<a href="https://colab.research.google.com/github/ShrutiKharate/K-Means-vs.-Hierarchical-K-Means-clustering-/blob/main/K_Means_vs_Hierarchical_K_Means_clustering_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# K-Means vs. Hierarchical K-Means on Iris

### 🎯 Objective
To **cluster** the standardized Iris dataset using **flat K-Means** and **Bisecting K-Means** (Hierarchical), and formally **compare** the resulting cluster structures against each other and against the true species labels.

---

### 📦 Data & Preparation
* **Data:** Iris dataset (4 features: sepal/petal length/width).
* **Preparation:** All 4 features were **standardized** (zero mean, unit variance) before clustering.
* **Labels:** Species labels were used **only for external evaluation**, not for fitting.

---

### ⚙️ Methods & Selection
1.  **Flat K-Means:**
    * Run for $k=2$ through $k=8$.
    * **Selected $k=3$** based on the WCSS **Elbow** (significant drop) and alignment with the known species count, despite $k=2$ having the highest Silhouette score.
2.  **Bisecting K-Means:**
    * Implemented via a greedy top-down approach.
    * Required **2 splits** to reach the target of **3 leaf clusters** (matching species count).

---

### ✅ Evaluation Metrics
* **External Validity (Alignment with Species):** Measured using **Adjusted Rand Index (ARI)**, **Normalized Mutual Information (NMI)**, V-measure, Homogeneity, and Completeness.
* **Internal Validity (Quality):** Measured using **WCSS/SSE** and mean **Silhouette** score.
* **Structure:** Centroids were compared for interpretation across the two methods.

---

### ✨ Key Finding
For this specific implementation on the standardized Iris dataset, **K-Means ($k=3$) and Bisecting K-Means ($K=3$) produced identical cluster assignments**. This was shown by their perfectly matching ARI, NMI, and Silhouette scores (e.g., ARI $\approx 0.62$, Silhouette $\approx 0.46$).

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score, normalized_mutual_info_score, v_measure_score, homogeneity_score, completeness_score
from sklearn.decomposition import PCA
# For Bisecting K-Means, we'll use a custom function/loop

# --- A) PREPARATION ---
# 1. Load Data
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y_true = iris.target # True species labels (for evaluation only)

# 2. Standardize Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# --- B) K-MEANS (Flat) ITERATIONS ---
k_range = range(2, 9)
wcss_list = []
silhouette_list = []
# Optional: calinski_harabasz_list = []
# Optional: davies_bouldin_list = []

for k in k_range:
    # 1. Run K-Means
    kmeans_model = KMeans(n_clusters=k, init='k-means++', n_init=10, random_state=42)
    kmeans_model.fit(X_scaled)

    # 2. Store Metrics
    wcss_list.append(kmeans_model.inertia_)
    # Silhouette requires at least k=2
    if k > 1:
        silhouette_list.append(silhouette_score(X_scaled, kmeans_model.labels_))
    else:
        silhouette_list.append(np.nan)

# Create the Model Selection DataFrame (for Table 1)
model_selection_df = pd.DataFrame({
    'k': list(k_range),
    'WCSS': wcss_list,
    'Silhouette (mean)': silhouette_list
})

print("--- K-Means Model Selection Data ---")
print(model_selection_df.to_markdown(index=False))

--- K-Means Model Selection Data ---
|   k |     WCSS |   Silhouette (mean) |
|----:|---------:|--------------------:|
|   2 | 222.362  |            0.58175  |
|   3 | 139.82   |            0.459948 |
|   4 | 114.093  |            0.386941 |
|   5 |  90.9275 |            0.345901 |
|   6 |  81.5444 |            0.317079 |
|   7 |  72.6311 |            0.320197 |
|   8 |  62.5406 |            0.338692 |


In [2]:
# --- B) K-MEANS (Final Model) ---
# ASSUME k_final = 3 based on typical Iris results (you should choose this based on your plots)
k_final = 3

# Refit K-Means
kmeans_final = KMeans(n_clusters=k_final, init='k-means++', n_init=10, random_state=42)
kmeans_final.fit(X_scaled)

# Save Assignments and Centroids (for Table 2)
kmeans_labels = kmeans_final.labels_
kmeans_centroids_scaled = kmeans_final.cluster_centers_

# --- C) BISECTING K-MEANS (Custom Implementation) ---

# Function to calculate SSE for a given cluster/data subset
def calculate_sse(data):
    if len(data) == 0:
        return 0
    return np.sum(np.sum((data - np.mean(data, axis=0))**2))

# Initial state
current_clusters = [np.arange(len(X_scaled))] # Start with one cluster (all indices)
bisecting_trace = []

# Loop to reach K=3 leaves (requires 2 splits)
for step in range(1, 3): # We need 2 steps to go from 1 -> 2 -> 3 clusters
    # 1. Find the cluster with the largest SSE
    sse_list = [(i, calculate_sse(X_scaled[indices])) for i, indices in enumerate(current_clusters)]
    largest_sse_index, parent_sse = max(sse_list, key=lambda item: item[1])
    parent_indices = current_clusters.pop(largest_sse_index)

    # 2. Split the cluster (using K-Means k=2)
    kmeans_split = KMeans(n_clusters=2, init='k-means++', n_init=10, random_state=42)
    kmeans_split.fit(X_scaled[parent_indices])
    split_labels = kmeans_split.labels_

    # 3. Create the two new child clusters
    child1_indices = parent_indices[split_labels == 0]
    child2_indices = parent_indices[split_labels == 1]

    child1_sse = calculate_sse(X_scaled[child1_indices])
    child2_sse = calculate_sse(X_scaled[child2_indices])

    delta_sse = parent_sse - (child1_sse + child2_sse)

    # Add new clusters to the list
    current_clusters.extend([child1_indices, child2_indices])

    # Record trace (for Table 4)
    bisecting_trace.append({
        'Step': step,
        'Parent SSE': parent_sse,
        'Child SSE (1)': child1_sse,
        'Child SSE (2)': child2_sse,
        'ΔSSE': delta_sse
    })

# Final Bisecting K-Means Results
bisecting_labels = np.zeros(len(X_scaled), dtype=int)
bisecting_centroids_scaled = []
for i, indices in enumerate(current_clusters):
    bisecting_labels[indices] = i
    bisecting_centroids_scaled.append(np.mean(X_scaled[indices], axis=0))
bisecting_centroids_scaled = np.array(bisecting_centroids_scaled)

print("\n--- Bisecting Splits (Trace Data) ---")
print(pd.DataFrame(bisecting_trace).to_markdown(index=False))


--- Bisecting Splits (Trace Data) ---
|   Step |   Parent SSE |   Child SSE (1) |   Child SSE (2) |     ΔSSE |
|-------:|-------------:|----------------:|----------------:|---------:|
|      1 |      600     |        174.693  |         47.6684 | 377.638  |
|      2 |      174.693 |         44.3834 |         47.7687 |  82.5412 |


In [5]:
# --- D) EVALUATION ---

# Calculate Metrics for both models
metrics = {
    'Method': [], 'ARI': [], 'NMI': [], 'V-measure': [],
    'Homogeneity': [], 'Completeness': [], 'Silhouette': [], 'WCSS': []
}

# K-Means Metrics
metrics['Method'].append(f'K-Means (k={k_final})')
metrics['ARI'].append(adjusted_rand_score(y_true, kmeans_labels))
metrics['NMI'].append(normalized_mutual_info_score(y_true, kmeans_labels))
metrics['V-measure'].append(v_measure_score(y_true, kmeans_labels))
metrics['Homogeneity'].append(homogeneity_score(y_true, kmeans_labels))
metrics['Completeness'].append(completeness_score(y_true, kmeans_labels))
metrics['Silhouette'].append(silhouette_score(X_scaled, kmeans_labels))
metrics['WCSS'].append(kmeans_final.inertia_)



In [6]:
# Bisecting K-Means Metrics
total_bisecting_sse = sum(calculate_sse(X_scaled[indices]) for indices in current_clusters)

metrics['Method'].append('Bisecting (K=3)')
metrics['ARI'].append(adjusted_rand_score(y_true, bisecting_labels))
metrics['NMI'].append(normalized_mutual_info_score(y_true, bisecting_labels))
metrics['V-measure'].append(v_measure_score(y_true, bisecting_labels))
metrics['Homogeneity'].append(homogeneity_score(y_true, bisecting_labels))
metrics['Completeness'].append(completeness_score(y_true, bisecting_labels))
metrics['Silhouette'].append(silhouette_score(X_scaled, bisecting_labels))
metrics['WCSS'].append(total_bisecting_sse)



In [7]:
# Create the Metrics DataFrame (for Table 3)
metrics_df = pd.DataFrame(metrics).drop(columns=['WCSS'])
print("\n--- External/Internal Metrics Data ---")
print(metrics_df.to_markdown(index=False))





--- External/Internal Metrics Data ---
| Method          |      ARI |      NMI |   V-measure |   Homogeneity |   Completeness |   Silhouette |
|:----------------|---------:|---------:|------------:|--------------:|---------------:|-------------:|
| K-Means (k=3)   | 0.620135 | 0.659487 |    0.659487 |      0.659127 |       0.659848 |     0.459948 |
| Bisecting (K=3) | 0.620135 | 0.659487 |    0.659487 |      0.659127 |       0.659848 |     0.459948 |


In [8]:
# Centroids Formatting (for Tables 2 & 3)
feature_names = iris.feature_names



In [9]:
# K-Means Centroids
kmeans_centroids_df = pd.DataFrame(
    kmeans_centroids_scaled,
    columns=feature_names
).reset_index().rename(columns={'index': 'Cluster'})
print("\n--- Centroids (Final K-Means) ---")
print(kmeans_centroids_df.to_markdown(index=False))




--- Centroids (Final K-Means) ---
|   Cluster |   sepal length (cm) |   sepal width (cm) |   petal length (cm) |   petal width (cm) |
|----------:|--------------------:|-------------------:|--------------------:|-------------------:|
|         0 |          -0.0502199 |         -0.883376  |            0.347738 |           0.281527 |
|         1 |          -1.01458   |          0.853263  |           -1.30499  |          -1.25489  |
|         2 |           1.13597   |          0.0884217 |            0.996155 |           1.01753  |


In [10]:
# Bisecting K-Means Centroids
bisecting_centroids_df = pd.DataFrame(
    bisecting_centroids_scaled,
    columns=feature_names
).reset_index().rename(columns={'index': 'Leaf'})
bisecting_centroids_df['Leaf'] = ['A', 'B', 'C']
print("\n--- Centroids (Bisecting K-Means, K=3) ---")
print(bisecting_centroids_df.to_markdown(index=False))




--- Centroids (Bisecting K-Means, K=3) ---
| Leaf   |   sepal length (cm) |   sepal width (cm) |   petal length (cm) |   petal width (cm) |
|:-------|--------------------:|-------------------:|--------------------:|-------------------:|
| A      |          -1.01458   |          0.853263  |           -1.30499  |          -1.25489  |
| B      |          -0.0502199 |         -0.883376  |            0.347738 |           0.281527 |
| C      |           1.13597   |          0.0884217 |            0.996155 |           1.01753  |
